<a href="https://colab.research.google.com/github/emd-aquila/Xc51-MOFs/blob/main/MOF_screening/download_mofx_cifs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""Set up packages and target directory/API"""

import os
import requests
from tqdm import tqdm

# Set your target directory
MOF_DIR = "MOFX_CIFs"
os.makedirs(MOF_DIR, exist_ok=True)

# MOFX API endpoint
API_URL = "https://mof.tech.northwestern.edu/api/mof/"

In [ ]:
"""Query MOFX API for list of MOFs"""

response = requests.get(API_URL)
response.raise_for_status()
mof_entries = response.json()

print(f"Found {len(mof_entries)} MOFs in MOFX-DB.")

In [ ]:
"""Download CIF files for each MOF"""

for entry in tqdm(mof_entries):
    mof_id = entry["id"]
    mof_name = entry["name"]

    # Some MOF names may contain slashes or colons — clean up
    safe_name = mof_name.replace("/", "_").replace(":", "_")
    cif_path = os.path.join(MOF_DIR, f"{safe_name}.cif")

    # Skip already-downloaded files
    if os.path.exists(cif_path):
        continue

    cif_url = f"https://mof.tech.northwestern.edu/api/mof/{mof_id}/cif"

    try:
        cif_response = requests.get(cif_url)
        cif_response.raise_for_status()
        with open(cif_path, "wb") as f:
            f.write(cif_response.content)
    except Exception as e:
        print(f"[ERROR] Could not download {mof_name} (ID: {mof_id}): {e}")
